In [2]:
# ============================================================
# AutoTag-Engine — Automated Cloud Resource Tagging & Enforcement (Colab ready)
# ============================================================

import os
import json
import random
from datetime import datetime, timezone

# Configuration
RESOURCES_FILE = "data/resources.json"
TAG_REPORT_FILE = "data/tagging_report.json"

# Define required tag schema
REQUIRED_TAGS = {
    "owner": None,       # e.g., team or person
    "environment": ["dev", "test", "prod"],
    "cost_center": None  # business cost center
}

# Ensure directories
os.makedirs(os.path.dirname(RESOURCES_FILE), exist_ok=True)
os.makedirs(os.path.dirname(TAG_REPORT_FILE), exist_ok=True)

# Generate dummy resource inventory if not exists
if not os.path.exists(RESOURCES_FILE):
    resources = []
    now = datetime.now(timezone.utc)
    for i in range(1, 41):
        resources.append({
            "resource_id": f"res-{i:03}",
            "type": random.choice(["vm_instance", "storage_bucket", "database", "lambda_function"]),
            "tags": {
                # randomly include some or omit
                "owner": random.choice([f"team{random.randint(1,5)}", None]),
                "environment": random.choice(["dev", "test", "prod", None]),
                "cost_center": random.choice([f"CC{random.randint(100,999)}", None])
            },
            "created_at": now.isoformat()
        })
    with open(RESOURCES_FILE, "w", encoding="utf-8") as f:
        json.dump(resources, f, indent=2)

def load_resources(path):
    with open(path, "r", encoding="utf-8") as f:
        return json.load(f)

def check_tags(resource, required_tags):
    missing = []
    invalid = []
    for tag_key, allowed in required_tags.items():
        value = resource.get("tags", {}).get(tag_key)
        if value is None:
            missing.append(tag_key)
        else:
            if isinstance(allowed, list):
                if value not in allowed:
                    invalid.append(f"{tag_key}={value}")
    return missing, invalid

def run_tag_audit(input_path, output_path, required_tags):
    resources = load_resources(input_path)
    report = {
        "timestamp": datetime.now(timezone.utc).isoformat(),
        "total_resources": len(resources),
        "compliant": 0,
        "non_compliant": 0,
        "details": []
    }
    for res in resources:
        missing, invalid = check_tags(res, required_tags)
        if not missing and not invalid:
            report["compliant"] += 1
        else:
            report["non_compliant"] += 1
            report["details"].append({
                "resource_id": res["resource_id"],
                "type": res["type"],
                "missing_tags": missing,
                "invalid_tags": invalid,
                "tags": res.get("tags", {})
            })
    with open(output_path, "w", encoding="utf-8") as f:
        json.dump(report, f, indent=2)
    return report

if __name__ == "__main__":
    tag_report = run_tag_audit(RESOURCES_FILE, TAG_REPORT_FILE, REQUIRED_TAGS)
    print("✅ Tagging audit complete:", tag_report)


✅ Tagging audit complete: {'timestamp': '2025-10-26T09:51:24.440931+00:00', 'total_resources': 40, 'compliant': 7, 'non_compliant': 33, 'details': [{'resource_id': 'res-001', 'type': 'lambda_function', 'missing_tags': ['owner'], 'invalid_tags': [], 'tags': {'owner': None, 'environment': 'prod', 'cost_center': 'CC884'}}, {'resource_id': 'res-002', 'type': 'storage_bucket', 'missing_tags': ['environment'], 'invalid_tags': [], 'tags': {'owner': 'team2', 'environment': None, 'cost_center': 'CC903'}}, {'resource_id': 'res-003', 'type': 'vm_instance', 'missing_tags': ['owner'], 'invalid_tags': [], 'tags': {'owner': None, 'environment': 'prod', 'cost_center': 'CC225'}}, {'resource_id': 'res-005', 'type': 'database', 'missing_tags': ['owner', 'cost_center'], 'invalid_tags': [], 'tags': {'owner': None, 'environment': 'test', 'cost_center': None}}, {'resource_id': 'res-006', 'type': 'vm_instance', 'missing_tags': ['cost_center'], 'invalid_tags': [], 'tags': {'owner': 'team1', 'environment': 'pro